In [ ]:
import os
import sys
from pathlib import Path

import datarobot as dr
from dotenv import load_dotenv

# The notebook should be executed from the project root directory
if "_correct_path" not in locals():
    os.chdir("..")
    sys.path.append(".")
    print(f"changed dir to {Path('.').resolve()})")
    _correct_path = True
load_dotenv()
client = dr.Client()

In [ ]:
import textwrap

from infra.settings_main import project_name
from nbo.custom_metrics import metrics_manager
from nbo.i18n import gettext
from nbo.schema import (
    AppDataScienceSettings,
    AppInfraSettings,
    LLMModelSpec,
    OutcomeDetail,
    association_id,
)

custom_metric_baselines = metrics_manager.get_baseline_values()

In [ ]:
app_ds_settings = AppDataScienceSettings(
    page_title=gettext("Next Best Offer Email Campaign"),
    page_subtitle=textwrap.dedent(
        gettext("""\
            Combine predictive and generative AI to help automatically create highly personalized emails for your customers.
            All you need to do is select a customer from the dropdown and hit submit!""")
    ),
    record_identifier={
        "column_name": "Cust_ID",
        "display_name": gettext("Customer"),
    },
    text_explanation_feature="tariff_plan_conds",  # Optional, include when a text variable is present and ngrams feature explanations are desired
    no_text_gen_label="No action",  # Optional, include in predictions where an email is not desired
    default_number_of_explanations=3,
    target_probability_description=gettext(
        "the likelihood this customer is expected to purchase"
    ),  # Preceeded in prompt by "Feature is increasing/decreasing"
    email_prompt=textwrap.dedent(
        gettext("""\
            Draft an email to an existing client that markets the plan {prediction_label}.
            The email should be addressed to {selected_record} and contain a subject line and body.
            Be sure to only incorporate the following call plan details:
            {outcome_description}
            Keep the email tone {tone}. Keep the length {verbosity}. Try not to use many emojis.
            Incorporate the following list of factors that influenced our offer decision:

            {rsp}"""),
    ),
    outcome_details=[  # Each item should be a target value with it's corresponding label and optional description
        OutcomeDetail(
            prediction="Daytime plan",
            label=gettext("Daytime Plan"),
            description=textwrap.dedent(
                gettext("""\
                    Introducing our new Daytime Plan, tailored for professionals who primarily use their mobile services
                    during traditional working hours. The Daytime Plan offers unlimited calls, texts, and a robust 10GB of high-speed data
                    from 8 AM to 6 PM, Monday through Friday. Outside of these hours and on weekends, a cost-efficient pay-as-you-go rate
                    will be applied to give you the flexibility you need without breaking the bank. The plan also includes free voicemail
                    services and call forwarding options, making it an ideal choice for those who want to stay connected without the hassle
                    of worrying about overages during their most active hours. Whether you're coordinating with your team, staying in touch
                    with clients, or managing remote operations, our Daytime Plan ensures you're always just a call or text away.""")
            ),
        ),
        OutcomeDetail(
            prediction="Evening call plan",
            label=gettext("Evening Call Plan"),
            description=textwrap.dedent(
                gettext("""\
                    Introducing the Evening Call Plan, designed for individuals who are most active during the evening hours and are looking
                    for a cost-effective way to stay connected. This plan provides unlimited calls and texts, along with a generous 8GB of
                    high-speed data, all available from 6 PM to 12 AM every day. During daytime hours, a reasonable pay-as-you-go rate applies,
                    giving you the freedom to use your phone as needed without unexpected costs. Also included in this package are essential
                    features like free voicemail and call forwarding, making it a perfect fit for those who love socializing, gaming, or
                    streaming content in the evening. The Evening Call Plan offers the ideal solution for night owls, allowing them to
                    communicate and surf the web to their heart's content during the hours they are most active.""")
            ),
        ),
        OutcomeDetail(
            prediction="International call plan",
            label=gettext("International Call Plan"),
            description=textwrap.dedent(
                gettext("""\
                    Introducing our International Call Plan, specifically designed for the globally connected individual.
                    With this plan, enjoy unlimited international calls and texts to over 50 selected countries, ensuring you stay connected
                    with family, friends, and colleagues across borders without worrying about hefty fees. This plan also includes 15GB of
                    high-speed data, and unlimited domestic calls and texts, providing a comprehensive package for all your communication needs.
                    For destinations not included in our selected list, competitive per-minute rates apply, and we've also thrown in premium
                    features like free voicemail and call forwarding at no additional cost. The International Call Plan is perfect for
                    expatriates, frequent travelers, or anyone who wants to maintain seamless international connections without the burden of
                    restrictive costs.""")
            ),
        ),
        OutcomeDetail(
            prediction="Enhanced voicemail plan",
            label=gettext("Enhanced Voicemail Plan"),
            description=textwrap.dedent(
                gettext("""\
                    Introducing the Enhanced Voicemail Plan, the latest offering tailored for individuals seeking a cutting-edge
                    solution to their messaging needs. Recognizing the importance of voicemail in modern communication, our plan goes beyond just
                    storing messages. It features transcription services, turning your voicemails into readable text sent directly to your email or
                    preferred messaging platform. Additionally, users benefit from extended storage capacities, allowing voicemails to be saved for
                    up to 6 months, and priority playback options to streamline the most important messages. Integrated with AI-driven spam filters,
                    this plan ensures you only receive messages that truly matter, reducing clutter and enhancing productivity. Whether you're a busy
                    professional or someone who values organized and efficient communication, our Enhanced Voicemail Plan is designed to keep you
                    ahead in the digital age.""")
            ),
        ),
        OutcomeDetail(
            prediction="No action",
            label=gettext("No Action"),
            description="",
        ),
    ],
    custom_metric_baselines=custom_metric_baselines,
    association_id_column_name=association_id,
    tones=[
        gettext("authoritative and expert"),
        gettext("educational and informative"),
        gettext("formal and elevated"),
        gettext("friendly and casual"),
        gettext("lighthearted and funny"),
        gettext("witty and playful"),
    ],
    verbosity=[
        gettext("short and sweet"),
        gettext("normal"),
        gettext("long and detailed"),
    ],
    system_prompt=textwrap.dedent(
        gettext("""\
            You are a marketing specialist named Justin and you work for TelCorp., a telecommunications company.
            You have detailed information about each of your customers and their past purchasing and usage behavior.
            You also have a list of important factors that explain why a customer may enjoy a particular call plan offer.
            The goal is to incorporate these factors into a marketing email introducing a specific offer."""),
    ),
    model_spec=LLMModelSpec(
        input_price_per_1k_tokens=0.001,  # update these values based on the model's pricing
        output_price_per_1k_tokens=0.002,
    ),
)

In [ ]:
from datarobot_pulumi_utils.schema.training import (
    AdvancedOptionsArgs,
    AnalyzeAndModelArgs,
    AutopilotRunArgs,
)

use_case_name = f"NBO Next Best Offer [{project_name}]"
use_case_description = "Next best offer"

dataset_name = f"NBO Training Data [{project_name}]"
file_path = "assets/nbo_training.csv"

autopilotrun_args = AutopilotRunArgs(
    name=f"NBO AutoPilot Run [{project_name}]",
    analyze_and_model_config=AnalyzeAndModelArgs(
        metric="LogLoss",
        mode="quick",
        target="Offers",
    ),
    advanced_options_config=AdvancedOptionsArgs(
        seed=42,
    ),
)

registered_model_name = f"NBO [{project_name}]"

In [ ]:
from datarobotx.idp.autopilot import get_or_create_autopilot_run
from datarobotx.idp.datasets import get_or_create_dataset_from_file
from datarobotx.idp.registered_model_versions import (
    get_or_create_registered_leaderboard_model_version,
)
from datarobotx.idp.use_cases import get_or_create_use_case

In [ ]:
print(gettext("Creating Use Case {use_case_name}").format(use_case_name=use_case_name))

if "DATAROBOT_DEFAULT_USE_CASE" in os.environ:
    use_case_id = os.environ["DATAROBOT_DEFAULT_USE_CASE"]
else:
    use_case_id = get_or_create_use_case(
        endpoint=client.endpoint,
        token=client.token,
        name=use_case_name,
        description=use_case_description,
    )

In [ ]:
print(gettext("Creating Dataset {dataset_name}").format(dataset_name=dataset_name))
dataset_id = get_or_create_dataset_from_file(
    token=client.token,
    endpoint=client.endpoint,
    name=dataset_name,
    file_path=file_path,
    use_cases=use_case_id,
)

In [ ]:
print(gettext("Creating Autopilot Run {name}").format(name=autopilotrun_args.name))
project_id = get_or_create_autopilot_run(
    token=client.token,
    endpoint=client.endpoint,
    dataset_id=dataset_id,
    use_case=use_case_id,
    **autopilotrun_args.model_dump(mode="json"),
)

In [ ]:
recommended_model_id = dr.ModelRecommendation.get(project_id).model_id  # type: ignore[union-attr,attr-defined]

try:
    model = dr.Model.get(project_id, recommended_model_id)  # type: ignore[attr-defined]
    prediction_threshold = model.get_roc_curve(
        source="validation"
    ).get_best_f1_threshold()
except Exception:
    prediction_threshold = None

In [ ]:
import pandas as pd

print(
    gettext("Best model: {model_type}\n\nMetrics:").format(model_type=model.model_type)
)

pd.DataFrame.from_records(model.metrics)

In [ ]:
print(gettext("Creating Registered Model Version..."))
registered_model_version_id = get_or_create_registered_leaderboard_model_version(
    token=client.token,
    endpoint=client.endpoint,
    model_id=recommended_model_id,
    registered_model_name=registered_model_name,
    prediction_threshold=prediction_threshold,
)

In [ ]:
app_infra_settings = AppInfraSettings(
    registered_model_name=registered_model_name,
    registered_model_version_id=registered_model_version_id,
    scoring_dataset_id=dataset_id,
    use_case_id=use_case_id,
    project_id=project_id,
)

In [ ]:
import yaml

from infra.settings_main import (
    model_training_output_ds_settings,
    model_training_output_infra_settings,
)

with open(model_training_output_ds_settings, "w") as f:
    yaml.safe_dump(app_ds_settings.model_dump(mode="json"), f, allow_unicode=True)
with open(model_training_output_infra_settings, "w") as f:
    yaml.safe_dump(app_infra_settings.model_dump(mode="json"), f, allow_unicode=True)